# Упражнение 05 : Оптимизация Pandas

In [104]:
import pandas as pd

## прочитайте файл fines.csv, который вы сохранили в предыдущем упражнении

In [105]:
fines = pd.read_csv('../data/fines.csv')

## итерации: во всех следующих подзадачах вам нужно вычислить штрафы/возврат*год для каждой строки и создать новый столбец с вычисленными данными, а также измерить время с помощью магической команды %%timeit в ячейке

* цикл: напишите функцию, которая перебирает фрейм данных с помощью for i in range(0, len(df)), iloc и append() для добавления в список. Присвойте результат работы функции новому столбцу в фрейме данных

In [106]:
%%timeit -n1 -r1 -p4
def func_loop():
    fin_ref_year = []
    for i in range(0, len(fines)):
        fin_ref_year.append(fines['Fines'].iloc[i] / fines['Refund'].iloc[i] * fines['Year'].iloc[i])
    return fin_ref_year

fines['fin_ref_year'] = func_loop()

14.94 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


* сделайте это с помощью iterrows()

In [107]:
%%timeit -n1 -r1 -p4
def func_iter(fines):
    fin_ref_year = []
    for i, row in fines.iterrows():
        fin_ref_year.append(row['Fines'] / row['Refund'] * row['Year'])
    return fin_ref_year

fines['fin_ref_year'] = func_iter(fines)

42.67 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


* сделайте это с помощью apply() и лямбда-функции

In [108]:
%%timeit -n1 -r1 -p4
fines['fin_ref_year'] = fines.apply(lambda row: row['Fines'] / row['Refund'] * row['Year'], axis=1)

7.217 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


* сделайте это, используя объекты серии из фрейма данных

In [109]:
%%timeit -n1 -r1 -p4
def func_series(fines):
    fines['fin_ref_year'] = fines['Fines'] / fines['Refund'] * fines['Year']

func_series(fines)

412.8 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


* сделайте это, как в предыдущей подзадаче , но с помощью метода .values

In [110]:
%%timeit -n1 -r1 -p4
def func_values(fines):
    fines['fin_ref_year'] = fines['Fines'].values / fines['Refund'].values * fines['Year'].values

func_values(fines)

272.9 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## индексирование: измерьте время с помощью волшебной команды %%timeit в ячейке

* получить строку для конкретного номера автомобиля, например, «O136HO197RUS»

In [111]:
%%timeit -n1 -r1 -p4
fines.loc[fines['CarNumber'] == 'O136HO197RUS']

597.9 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


* установите индекс в вашем фрейме данных с помощью CarNumber
* опять же, получите строку с тем же номером

In [112]:
fines.set_index('CarNumber', inplace=True)

In [113]:
%%timeit -n1 -r1 -p4
fines.loc[fines.index == 'O136HO197RUS']

425.2 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## понижающий коэффициент:

* запустите df.info(memory_usage=’deep’), обратите внимание на тип данных и использование памяти

In [114]:
fines.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to Р978ЕА10RUS
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Refund        930 non-null    int64  
 1   Fines         930 non-null    float64
 2   Make          930 non-null    object 
 3   Model         919 non-null    object 
 4   Year          930 non-null    int64  
 5   fin_ref_year  930 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 182.1 KB


In [115]:
fines

,Refund,Fines,Make,Model,Year,fin_ref_year
CarNumber,,,,,,
Y163O8161RUS,2,3200.0,Ford,Focus,1989,3182400.0
E432XX77RUS,1,6500.0,Toyota,Camry,1995,12967500.0
7184TT36RUS,1,2100.0,Ford,Focus,1984,4166400.0
X582HE161RUS,2,2000.0,Ford,Focus,2015,2015000.0
92918M178RUS,1,5700.0,Ford,Focus,2014,11479800.0
...,...,...,...,...,...,...
С735АК10RUS,1,1000.0,Volkswagen,Jetta,2000,2000000.0
О504НК89RUS,2,2000.0,Volkswagen,Touareg,2004,2004000.0
Е157ВХ05RUS,1,1000.0,Ford,Mondeo,2006,2006000.0


* сделайте копию () вашего исходного фрейма данных в другой оптимизированный фрейм данных

In [116]:
fines_copy = fines.copy()

fines_copy['Refund'] = pd.to_numeric(fines_copy['Refund'], downcast='integer')
fines_copy['Year'] = pd.to_numeric(fines_copy['Year'], downcast='integer')
fines_copy['Fines'] = pd.to_numeric(fines_copy['Fines'], downcast='float')
fines_copy['fin_ref_year'] = pd.to_numeric(fines_copy['fin_ref_year'], downcast='float')

fines_copy['Make'] = fines_copy['Make'].astype('category')
fines_copy['Model'] = fines_copy['Model'].astype('category')
fines_copy.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to Р978ЕА10RUS
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Refund        930 non-null    int8    
 1   Fines         930 non-null    float32 
 2   Make          930 non-null    category
 3   Model         919 non-null    category
 4   Year          930 non-null    int16   
 5   fin_ref_year  930 non-null    float64 
dtypes: category(2), float32(1), float64(1), int16(1), int8(1)
memory usage: 72.3 KB
